# Python Doesn't Have Good Numeric Support
* Python integers are actually an object with header and typing information
* access to Python integers requires a level of indirection
* In C, integers are directly accessible in memory without indirection


## The Problem is Even Worse for Python Lists
* Python lists are immensely flexible
  * no fixed size
  * OK to have heterogeneous data
* ...but as a result they are not likely to be contiguous in memory
* and even if they are, there is still a lot of indirection required
* so they aren't good for fast number crunching


In [ ]:
pylist = list(range(1_000_000))
%timeit [i + 1 for i in pylist]

## One solution is to use PyTorch tensors
* written in C++
* allows for vectorized operations

In [ ]:
import torch as pt
pt.__version__

## PyTorch Scalars

In [ ]:
pt.tensor([42, 12])

In [ ]:
pt.tensor(42).dtype

In [ ]:
pt.tensor([[42, 12], [12, 3]])

In [ ]:
len(pt.tensor(42).shape) == 0

In [ ]:
pt.tensor(3.14).dtype

In [ ]:
pt.tensor(3.14).item()

In [ ]:
pt.tensor(3.14).item() == 3.14

## IEEE Standard for Floating-Point Arithmetic (IEEE 754)
* a refresher on floating point precision issues

In [ ]:
x = 0.3
x

In [ ]:
3 * 0.1 == x

In [ ]:
3 * 0.1

In [ ]:
x = pt.tensor(3.14)

In [ ]:
x.to(pt.uint8).item()

In [ ]:
pt.tensor(x.to(pt.uint8))

In [ ]:
pt.tensor(x.to(pt.uint8).item(), dtype = pt.float32)

## __pt.trunc()__

* nearest integer __`i`__ which is closer to zero than __`x`__ is

In [ ]:
pt.trunc(x)

In [ ]:
pt.trunc(pt.tensor(2.01))

## __pt.floor()__

* the largest integer __`i`__, such that __`i <= x`__

In [ ]:
pt.floor(x)

In [ ]:
pt.floor(pt.tensor(2.01))

In [ ]:
pt.floor(pt.tensor(2.))

In [ ]:
pt.floor(pt.tensor(-3.14))

## __pt.ceil()__

* the smallest integer __`i`__, such that __`i >= x`__

In [ ]:
pt.ceil(x)

In [ ]:
pt.ceil(pt.tensor(2.01))

In [ ]:
pt.ceil(pt.tensor(2.))

* can __pt.ceil()__ be used in place of __pt.floor()__ ?

In [ ]:
pt.ceil(x) - 1

In [ ]:
pt.ceil(pt.tensor(2.01)) - 1

In [ ]:
pt.ceil(pt.tensor(2.)) - 1

## PyTorch arrays
* data is stored contiguously in memory

In [ ]:
# pytorch will infer the data type
a = pt.tensor([1, 4, 2, 5, 3])
a, a.dtype

In [ ]:
a = pt.tensor([3.14, 4, 2, 3])
a, a.dtype

In [ ]:
# ...or you can be explicit
a = pt.tensor([1, 2, 3, 4], dtype=pt.float32)
a

In [ ]:
pt.tensor([range(i, i + 3) for i in [2, 4, 6]])

In [ ]:
pt.zeros(10, dtype=int)

In [ ]:
pt.ones((3, 5), dtype=float)

In [ ]:
pt.eye(5)  # pt.tensor(np.eye(5))

In [ ]:
pt.full((3, 5), 42, dtype=int)

In [ ]:
pt.arange(0, 20, 2)

In [ ]:
pt.linspace(0, 1, 5)

## Pseudo-Random Numbers

In [ ]:
pt.manual_seed(0);

In [ ]:
pt.randn(3, 3, 4)

In [ ]:
pt.normal(0, 1, size = (3, 3))


In [ ]:
pt.randint(0, 10, (3, 3))

## Converting array types

In [ ]:
x = pt.linspace(0, 10, 50)
x

In [ ]:
x.to(int)

## Multi-dimensional Arrays

In [ ]:
x2 = pt.randint(10, size=[3, 4])
x2

## True "matrix-style" indexing

In [ ]:
x2[0, 0]

In [ ]:
x2[2, 0]

In [ ]:
x2[2, -1]

In [ ]:
x2[0, 0] = 12
x2

In [ ]:
pt.arange(0, 9).reshape(3, 3)

## Array Slicing

In [ ]:
x = pt.arange(10)
x[:5]

In [ ]:
x[5:]

In [ ]:
x[4:7]

In [ ]:
x[::2]

In [ ]:
x[1::2]

In [ ]:
x[::-1] # :)

In [ ]:
reversed(x)

In [ ]:
reversed(x)[5::2]

## Filtering 1-dimensional data

In [ ]:
x = pt.tensor([ 1, 0, 5, 2, 1, 0, 8, 0, 0 ])

In [ ]:
x.nonzero()

In [ ]:
x[x.nonzero()]

In [ ]:
x[x < 3]

## Filtering 2-dimensional data

In [ ]:
x = pt.tensor([[1, 0, 0], [0, 5, 0], [7, 8, 0]])
x

In [ ]:
# produces two arrays, one with x coords, one with y coords
x.nonzero()

In [ ]:
x.nonzero(as_tuple = True)

In [ ]:
x[x.nonzero(as_tuple = True)]

In [ ]:
y = pt.arange(1, 10).reshape(3, 3)
y

In [ ]:
y.index_select(dim = 0, index = pt.tensor([0, 2]))

In [ ]:
y.index_select(dim = 1, index = pt.tensor([0, 2]))

In [ ]:
y.triu()

In [ ]:
y.tril()

In [ ]:
y.tril().T #transpose

## Multi-dimensional subarrays

In [ ]:
x2

In [ ]:
x2[:2, :3]

In [ ]:
x2[:2, ::2]

In [ ]:
x2[::-1, ::-1]

In [ ]:
x2

In [ ]:
x2.numel()

In [ ]:
indices = pt.arange(x2.numel() - 1, -1, -1)
indices

In [ ]:
x2

In [ ]:
pt.take(x2, indices).reshape(x2.shape) #x2[::-1, ::-1]

## Subarray Views

In [ ]:
x2, id(x2)

In [ ]:
x2_sub = x2[:2, :2]
x2_sub, id(x2_sub)

In [ ]:
x2_sub[0, 0] = 99
x2_sub, id(x2_sub)

In [ ]:
x2, id(x2) # changes x2 as well, since the subarray has references to the original

## PyTorch ATen Functions
* operate on tensors as on contiguous blobs of data in memory
* _vectorized_ wrapper for a function that takes a fixed number of specific inputs and produces a fixed number of specific outputs

| Operator | ATen            | Description                         |
|----------|-----------------|-------------------------------------|
|   +      | pt.add          | Addition (e.g., 1 + 1 = 2)          |
|   -      | pt.subtract     | Subtraction (e.g., 3 - 2 = 1)       |
|   -      | pt.negative     | Unary negation (e.g., -2)           |
|   *      | pt.multiply     | Multiplication (e.g., 2 * 3 = 6)    |
|   /      | pt.divide       | Division (e.g., 3 / 2 = 1.5)        |
|   //     | pt.floor_divide | Floor division (e.g., 3 // 2 = 1)   |
|   **     | pt.power        | Exponentiation (e.g., 2 ** 3 = 8)   |
|   %      | pt.mod          | Modulus/remainder (e.g., 9 % 4 = 1) |

## Vectorized Operations

In [ ]:
pytorch = pt.arange(1_000_000)
%timeit pytorch + 1

In [ ]:
x = pt.arange(9).reshape((3, 3))
2 ** x

In [ ]:
x = pt.arange(4)
-(0.5 * x + 1) ** 2

## Exponents and Logarithms

In [ ]:
x = pt.tensor([1., 2., 3.])
pt.exp(x)

In [ ]:
pt.pow(3, x)

In [ ]:
pt.log(pt.tensor([1., 2., 3.]))

In [ ]:
pt.log2(pt.tensor([1., 256., 65536.]))

In [ ]:
pt.log10(pt.tensor([1_000., 1_000_000., 10. ** 10]))

## Aggregations

In [ ]:
x = pt.arange(15).reshape(3, 5)
x

In [ ]:
x.sum()

In [ ]:
x.sum(dim = 1)

In [ ]:
x.sum(dim = 1, keepdims = True)

In [ ]:
x.sum(dim = 1)

In [ ]:
x.to(float).mean().item(), x.to(float).std().item()